In [ ]:
#Author: Christopher Couture Del Valle
#ECE 595 Z2

In [1]:
#pip install split-folders
!pip install --quiet optuna

In [2]:
# import dataset, model and layers  
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import cv2
import random
import matplotlib.pyplot as plt
from scipy import misc
from __future__ import print_function
from tensorflow import keras
from tensorflow.keras import layers
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
#import splitfolders
from keras import Model
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import img_to_array, load_img
from keras.layers import ZeroPadding2D
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D,Dropout, Flatten, BatchNormalization, Concatenate, AveragePooling2D
from keras.callbacks import EarlyStopping, LearningRateScheduler

#For cifar 100
from keras.datasets import cifar100

from keras.layers.normalization import BatchNormalization

import optuna
from optuna.integration import KerasPruningCallback
from optuna.trial import TrialState

#batch_size = 64
#num_classes = 15
#epochs = 200

batch_size = 32
num_classes = 1000
epochs = 20
num_predictions = 20

data_augmentation = True

# input image dimensions
#img_rows, img_cols = 64, 64

In [ ]:
#device_name = tf.test.gpu_device_name()
#if device_name != '/device:GPU:0':
#  raise SystemError('GPU device not found')
#print('Found GPU at: {}'.format(device_name))

In [ ]:
#Get access to google drive directory
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Spli folders in 70/30 by category
#splitfolders.ratio(("/content/drive/My Drive/15-Scene"), output = "/content/drive/My Drive/15Data", seed = 1337, ratio=(0.7,0.3),group_prefix=None)

CIFAR100

In [3]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
#image_size =32

x_train shape: (50000, 32, 32, 3)
x_test shape: (10000, 32, 32, 3)
50000 train samples
10000 test samples


In [ ]:
# Function to load image and convert into array
def convert_img_to_array(dir_path):
    x = []
    y = []
    one_hot_lookups = np.eye(15)
    for category in sorted(os.listdir(dir_path)):
        for file in os.listdir(dir_path + category):
            img = load_img(dir_path + category + '/' + file,
                       target_size=(64, 64))  # Load image
            img = img_to_array(img)  # Convert image to array
            img = preprocess_input(img)  # Preprocessing on the image
            x.append(img)  # Append image to X_train
            y.append(np.reshape(one_hot_lookups[int(category)],
                                  [15]))  # Append category to y_train
    return x,y

In [ ]:
#Function to load image data
def load_data():
    train_dir_path = "/content/drive/My Drive/15Data/train/"
    x_train,y_train = convert_img_to_array(train_dir_path)
    
    test_dir_path = "/content/drive/My Drive/15Data/val/"
    x_test, y_test = convert_img_to_array(test_dir_path)
    
    x_train = np.asarray(x_train)
    x_test = np.asarray(x_test)
    y_train = np.asarray(y_train)
    y_test = np.asarray(y_test)
    
    x_train /= np.std(x_train, axis = 0)
    x_test /= np.std(x_test, axis = 0)
    
    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_ds = train_ds.shuffle(60000).batch(batch_size).take(3133)

    test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    test_ds = test_ds.shuffle(10000).batch(batch_size).take(1352)
    return train_ds, test_ds
#(x_train, y_train), (x_test, y_test) = load_data()

In [ ]:
#def get_custom():
    #(x_train, y_train), (x_test, y_test) = load_data()
    #x_train = x_train.astype('float32')
    #x_test = x_test.astype('float32')
    #x_train /= 255
    #x_test /= 255
    #y_train = y_train.astype("int32")
    #y_test = y_test.astype("int32")
    #test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    #test_ds = test_ds.shuffle(10000).batch(batch_size).take(1352)
    #return train_ds, test_ds

    

In [4]:
def create_model(trial):

  weight_decay = trial.suggest_float("weight_decay", 1e-10, 1e-3, log=True)

  model = Sequential()
  model.add(Conv2D(32, (3,3), padding='same', strides = 1, activation='relu', 
                   kernel_initializer='he_uniform', 
                   kernel_regularizer=keras.regularizers.l2(weight_decay), 
                   ))

  model.add(BatchNormalization())
  model.add(Conv2D(32, (3,3), padding='same', strides = 1, activation='relu', 
                   kernel_initializer='he_uniform', 
                   kernel_regularizer=keras.regularizers.l2(weight_decay))) 
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.2))
 
  model.add(Conv2D(64, (3,3), padding='same', strides = 1, activation='relu', 
                   kernel_initializer='he_uniform', 
                   kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(BatchNormalization())
  model.add(Conv2D(64, (3,3), padding='same', strides = 1, activation='relu', 
                   kernel_initializer='he_uniform', 
                   kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.3))

  model.add(Conv2D(128, (3,3), padding='same', strides = 1, activation='relu', 
                   kernel_initializer='he_uniform', 
                   kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(BatchNormalization())
  model.add(Conv2D(128, (3,3), padding='same', strides = 1, activation='relu', 
                   kernel_initializer='he_uniform', 
                   kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.4))

  model.add(Conv2D(256, (3,3), padding='same', strides = 1, activation='relu', 
                   kernel_initializer='he_uniform', 
                   kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.6))
 
  model.add(Flatten())
  model.add(Dense(num_classes, activation='softmax'))

  optimizer = create_optimizer(trial)
  model.compile(loss=keras.losses.categorical_crossentropy,optimizer=optimizer,metrics=['accuracy'])
 
  return model

In [5]:
def create_optimizer(trial):
    # We optimize the choice of optimizers as well as their parameters.
    kwargs = {}
    #optimizer_options = ["RMSprop", "Adam", "SGD"]
    optimizer_options = ["Adam"]
    optimizer_selected = trial.suggest_categorical("optimizer", optimizer_options)
    if optimizer_selected == "RMSprop":
        kwargs["learning_rate"] = trial.suggest_float(
            "rmsprop_learning_rate", 1e-5, 1e-1, log=True
        )
        kwargs["decay"] = trial.suggest_float("rmsprop_decay", 0.85, 0.99)
        kwargs["momentum"] = trial.suggest_float("rmsprop_momentum", 1e-5, 1e-1, log=True)
    elif optimizer_selected == "Adam":
        kwargs["learning_rate"] = trial.suggest_float("adam_learning_rate", 1e-5, 1e-1, log=True)
    elif optimizer_selected == "SGD":
        kwargs["learning_rate"] = trial.suggest_float(
            "sgd_opt_learning_rate", 1e-5, 1e-1, log=True
        )
        kwargs["momentum"] = trial.suggest_float("sgd_opt_momentum", 1e-5, 1e-1, log=True)

    optimizer = getattr(tf.optimizers, optimizer_selected)(**kwargs)
    return optimizer

CIFAR100

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
from keras.preprocessing.image import ImageDataGenerator
if not data_augmentation:
    print('Not using data augmentation.')
    history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)
   
    # Fit the model on the batches generated by datagen.flow().
    #history = model.fit_generator(datagen.flow(x_train, y_train,
         #                            batch_size=batch_size),
         #               epochs=epochs,
         #               validation_data=(x_test, y_test))

Using real-time data augmentation.


In [16]:
def objective(trial):
  #train_ds, test_ds = load_data()
  model = create_model(trial)
  history = model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                                    callbacks=[KerasPruningCallback(trial, "val_accuracy")],
                                    epochs=20,
                                    verbose=1,
                                    validation_data=(x_test, y_test))
  
  #history = model.fit(train_ds,
   #       batch_size=batch_size,
    #      callbacks=[KerasPruningCallback(trial, "val_accuracy")],
     #     epochs=epochs,
      #    verbose=1,
       #   validation_data=test_ds)

  score = model.evaluate(x_test,y_test, verbose = 0)
  
  return score[1]


In [17]:
if __name__ == "__main__":
    
    study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=3)
    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-07-21 23:41:04,344] A new study created in memory with name: no-name-65f3b28e-c025-4a94-9bb7-34197dafd53c
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning:

KerasPruningCallback has been deprecated in v2.1.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.1.0. Recent Keras release (2.4.0) simply redirects all APIs in the standalone keras package to point to tf.keras. There is now only one Keras: tf.keras. There may be some breaking changes for some workflows by upgrading to keras 2.4.0. Test before upgrading. REF:https://github.com/keras-team/keras/releases/tag/2.4.0

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



Epoch 1/20
1563/1563 [==============================] - 33s 19ms/step - loss: 5.2186 - accuracy: 0.0351 - val_loss: 3.6665 - val_accuracy: 0.1489
Epoch 2/20
1563/1563 [==============================] - 31s 20ms/step - loss: 3.8641 - accuracy: 0.1070 - val_loss: 3.3932 - val_accuracy: 0.1898
Epoch 3/20
1563/1563 [==============================] - 29s 18ms/step - loss: 3.5573 - accuracy: 0.1523 - val_loss: 3.1406 - val_accuracy: 0.2500
Epoch 4/20
1563/1563 [==============================] - 30s 19ms/step - loss: 3.3281 - accuracy: 0.1930 - val_loss: 2.9734 - val_accuracy: 0.2711
Epoch 5/20
1563/1563 [==============================] - 30s 19ms/step - loss: 3.1466 - accuracy: 0.2231 - val_loss: 2.7613 - val_accuracy: 0.3128
Epoch 6/20
1563/1563 [==============================] - 30s 19ms/step - loss: 3.0001 - accuracy: 0.2529 - val_loss: 2.6135 - val_accuracy: 0.3370
Epoch 7/20
1563/1563 [==============================] - 31s 20ms/step - loss: 2.8809 - accuracy: 0.2748 - val_loss: 2.5295 -

[I 2021-07-21 23:51:29,038] Trial 0 finished with value: 0.4982999861240387 and parameters: {'weight_decay': 4.7334492434534885e-09, 'optimizer': 'Adam', 'adam_learning_rate': 0.0004190581840410445}. Best is trial 0 with value: 0.4982999861240387.


Epoch 1/20
1563/1563 [==============================] - 34s 20ms/step - loss: 8.1662 - accuracy: 0.0101 - val_loss: 4.7787 - val_accuracy: 0.0445
Epoch 2/20
1563/1563 [==============================] - 29s 19ms/step - loss: 5.0253 - accuracy: 0.0235 - val_loss: 4.3554 - val_accuracy: 0.0744
Epoch 3/20
1563/1563 [==============================] - 30s 19ms/step - loss: 4.6410 - accuracy: 0.0370 - val_loss: 4.1060 - val_accuracy: 0.0935
Epoch 4/20
1563/1563 [==============================] - 30s 19ms/step - loss: 4.4254 - accuracy: 0.0479 - val_loss: 3.9307 - val_accuracy: 0.1121
Epoch 5/20
1563/1563 [==============================] - 30s 19ms/step - loss: 4.2636 - accuracy: 0.0603 - val_loss: 3.8135 - val_accuracy: 0.1319
Epoch 6/20
1563/1563 [==============================] - 31s 20ms/step - loss: 4.1309 - accuracy: 0.0714 - val_loss: 3.7081 - val_accuracy: 0.1445
Epoch 7/20
1563/1563 [==============================] - 30s 19ms/step - loss: 4.0402 - accuracy: 0.0859 - val_loss: 3.6374 -

[I 2021-07-22 00:01:48,833] Trial 1 finished with value: 0.24320000410079956 and parameters: {'weight_decay': 2.0192731109728288e-10, 'optimizer': 'Adam', 'adam_learning_rate': 3.2428684067564e-05}. Best is trial 0 with value: 0.4982999861240387.


Epoch 1/20
1563/1563 [==============================] - 34s 20ms/step - loss: 5.2412 - accuracy: 0.0139 - val_loss: 4.5249 - val_accuracy: 0.0177
Epoch 2/20
1563/1563 [==============================] - 30s 19ms/step - loss: 4.5796 - accuracy: 0.0139 - val_loss: 4.5305 - val_accuracy: 0.0157
Epoch 3/20
1563/1563 [==============================] - 30s 19ms/step - loss: 4.5822 - accuracy: 0.0134 - val_loss: 4.6109 - val_accuracy: 0.0100
Epoch 4/20
1563/1563 [==============================] - 29s 18ms/step - loss: 4.6049 - accuracy: 0.0108 - val_loss: 4.6103 - val_accuracy: 0.0100
Epoch 5/20
1563/1563 [==============================] - 30s 19ms/step - loss: 4.6120 - accuracy: 0.0097 - val_loss: 4.6099 - val_accuracy: 0.0100
Epoch 6/20
1563/1563 [==============================] - 29s 18ms/step - loss: 4.6114 - accuracy: 0.0103 - val_loss: 4.6116 - val_accuracy: 0.0100
Epoch 7/20
1563/1563 [==============================] - 30s 19ms/step - loss: 4.6120 - accuracy: 0.0094 - val_loss: 4.6115 -

[I 2021-07-22 00:11:46,613] Trial 2 finished with value: 0.009999999776482582 and parameters: {'weight_decay': 7.118519422137838e-09, 'optimizer': 'Adam', 'adam_learning_rate': 0.012829373229911437}. Best is trial 0 with value: 0.4982999861240387.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.4982999861240387
  Params: 
    weight_decay: 4.7334492434534885e-09
    optimizer: Adam
    adam_learning_rate: 0.0004190581840410445


In [18]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

plot_param_importances(study)
plot_intermediate_values(study)
plot_optimization_history(study)

In [19]:
import numpy as np
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training', 'validation'], loc = 'upper left')
plt.title('Training and validation accuracy')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training', 'validation'], loc = 'upper left')
plt.title('Training and validation loss')
plt.show()

NameError: ignored